In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    SparkSession.builder
    .appName("Curso de PySpark")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .getOrCreate()
)

In [4]:
spark

In [5]:
path = "C:/Users/marlos.barros/Cursos/pyspark_na_pratica/DATASETS/LOGINS.parquet"

In [8]:
df = spark.read.format("parquet").load(path).limit(5)

In [9]:
df.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- email: string (nullable = true)
 |-- senha: string (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_cadastro: date (nullable = true)
 |-- ipv4: string (nullable = true)
 |-- cor_favorita: string (nullable = true)
 |-- profissao: string (nullable = true)
 |-- telefone: string (nullable = true)



### Contagem por Cor Favorita
- Função: **groupBy()** agrupa o DataFrame pela coluna **"cor_favorita"** e retorna a **contagem de ocorrências** para cada **cor**.
- Exemplo: Se houver **10 registros** onde a cor favorita é **"azul"** e **5** onde é **"vermelho"**, o resultado será:
    - **Azul: 10**
    - **Vermelho: 5**

In [10]:
df.groupBy(F.col("cor_favorita")).count()

cor_favorita,count
Roxo,1
Ciano,1
Azul,1
Marrom,1
Laranja,1


### Contagem por Estado e Cor Favorita
- Função: Agrupa o DataFrame por **duas colunas**, **"estado"** e **"cor_favorita"**, e conta quantas vezes cada combinação ocorre.
- Exemplo: O resultado poderia ser:
    - **SP, Azul: 5**
    - **SP, Vermelho: 3**
    - **RJ, Azul: 2**

In [11]:
df.groupBy(F.col("estado"), F.col("cor_favorita")).count()

estado,cor_favorita,count
RR,Roxo,1
GO,Ciano,1
MG,Azul,1
AC,Marrom,1
AP,Laranja,1


### Soma da Contagem por Estado
- Função: Primeiro, **agrupa por "estado"** e **"cor_favorita"** e **conta as ocorrências**. Depois, **agrupa por "estado"** novamente e **faz a soma das contagens por estado**.
- Exemplo:
    - Se o estado **SP** tiver **5** pessoas que gostam de **azul** e **3** que gostam de **vermelho**, a soma será:
        - **SP: 8**
    - **RJ:** **2** (se houver apenas 2 pessoas no RJ)

In [12]:
df.groupBy(F.col("estado"), F.col("cor_favorita")).count().groupBy("estado").sum("count")

estado,sum(count)
RR,1
GO,1
MG,1
AC,1
AP,1


### Média da Contagem por Estado
- Função: Agrupa por **"estado"** e **"cor_favorita"**, conta as ocorrências e depois agrupa por **"estado"** e calcula a **média** das contagens.
- Exemplo:
    - Se **SP** tiver **5** para **azul** e **3** para **vermelho**, a média será:
        - **SP: (5 + 3) / 2 = 4**

In [13]:
df.groupBy(F.col("estado"), F.col("cor_favorita")).count().groupBy("estado").avg("count")

estado,avg(count)
RR,1.0
GO,1.0
MG,1.0
AC,1.0
AP,1.0


### Mínimo da Contagem por Estado
- Função: Agrupa por **"estado"** e **"cor_favorita"**, conta as ocorrências, e depois agrupa por **"estado"** e encontra o valor **mínimo das contagens**.
- Exemplo:
    - Se **SP** tiver **5** para **azul** e **3** para **vermelho**, o mínimo será:
       - **SP: 3**

In [14]:
df.groupBy(F.col("estado"), F.col("cor_favorita")).count().groupBy("estado").min("count")

estado,min(count)
RR,1
GO,1
MG,1
AC,1
AP,1


### Máximo da Contagem por Estado
- Função: Agrupa por **"estado"** e **"cor_favorita"**, conta as ocorrências, e depois agrupa por **"estado"** e encontra o **valor máximo das contagens**.
- Exemplo:
    - Se **SP** tiver **5** para **azul** e **3** para **vermelho**, o máximo será:
        - **SP: 5**

In [15]:
df.groupBy(F.col("estado"), F.col("cor_favorita")).count().groupBy("estado").max("count")

estado,max(count)
RR,1
GO,1
MG,1
AC,1
AP,1


### Contagem por Ano de Nascimento
- Função: Agrupa o DataFrame pelo **ano** extraído da coluna **"data_de_nascimento"** e conta quantas vezes cada ano aparece.
- Exemplo:
    - 1990: **10 registros**
    - 1985: **5 registros**

In [16]:
df.groupBy(F.year("data_de_nascimento")).count()

year(data_de_nascimento),count
2006,1
1992,1
1921,1
2021,1
1969,1
